In [4]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from Eunomia.preferences import *
from Eunomia.additive_functions import *
from Eunomia.alternatives import *
from Eunomia.sampling import *
from Eunomia.mcmc import *
from Eunomia.degree import *
from Eunomia.experiments import *
import torch
import torch.nn as nn
pyro.set_rng_seed(1)

In [5]:
class PhiFunction(nn.Module):
    def __init__(self, num_attributes):
        super(PhiFunction, self).__init__()
        # Initialize phi values as a learnable parameter
        self.phi_values = nn.Parameter(torch.randn(num_attributes, num_attributes, 2, 2))

    def forward(self, X, Y):
        phi_sum = 0
        for A in range(X.shape[0]):
            for B in range(Y.shape[0]):
                phi_sum += self.phi_values[A, B, X[A].item(), Y[B].item()]
        return phi_sum

In [3]:
class BinaryModel(nn.Module):
    def __init__(self, num_attributes):
        super(BinaryModel, self).__init__()
        self.weights = nn.Parameter(torch.randn(num_attributes))
        self.phi_function = PhiFunction(num_attributes)

    def forward(self, X, Y):
        weighted_sum = torch.sum(self.weights * (X - Y))
        phi_sum = self.phi_function(X, Y)
        return weighted_sum + phi_sum
